# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [6]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

### Вопрос аналитику
Какие оценки поставил пользователь №1 и в каком количестве?

Что сделаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

Как это сделать для всех пользователей сразу:

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


In [ ]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

### Фильтры и вычисления с помощью loc и iloc

In [10]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [ ]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

In [ ]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [ ]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

In [1]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

NameError: name 'log' is not defined

### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

In [ ]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

Можно и через loc

In [ ]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

Для замера времени сделаем эти операции для набор жанров

In [ ]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [4]:
import numpy as np

In [7]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [8]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [11]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16


In [12]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       18938 non-null  int64         
 1   visit_id        18938 non-null  object        
 2   url             18938 non-null  object        
 3   region          18938 non-null  object        
 4   user_id         18938 non-null  object        
 5   traffic_source  18938 non-null  object        
 6   date            18938 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.0+ MB


In [13]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32,14
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44,14
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55,14
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05,14
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16,14


### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [ ]:
from pymystem3 import Mystem

In [ ]:
search = 'курс гривны к рублю рубли рублях'

In [ ]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

In [ ]:
' '.join(lemmas)